In [1]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 10000)

# Algorithm
Define Group1, Group2 to compare between, and subpopulations as the factors for changes, and treatments as the resons for those factors.
1. Define DivExplorer to look only on the groups' data. The algorithm will bring the subpopulations with the support threshold, and also will calculate each group's average income.
2. For each subpopulation, will run search for finding the best treatment
3. Find the "best" K-group over all (subpopulation, treatment) we found

# SO Data

GROUP_1 = [{"att": "United States of America", "value": 1}]
GROUP_2 = [{"att": "United Kingdom of Great Britain and Northern Ireland", "value": 1}]

GROUPS = [{"att": "Gender", "value": "Woman"}, {"att": "Gender", "value": "Man"}, {"att": "Age", "value": "35-44 years old"}, {"att": "Age", "value": "25-34 years old"}
    ,{"att": "White", "value": 1}, {"att": "Asian", "value": 1}, {"att": "Black", "value": 1}]
    
df_so = [["Age,Gender,"Race","DevType","Education"]
1. Divexplorer - > df[USA or UK][[Age,Gender,Race]]

In [10]:
df = pd.read_csv("outputs/so/interesting_subpopulations.csv")
df

,support,itemset,GROUP_1,GROUP_2,ConvertedCompYearly,ConvertedCompYearly_group1,ConvertedCompYearly_group2,ConvertedCompYearly_div,length,support_count
0,0.755448,"frozenset({'Gender=Man', 'Asian=0', 'White=1'})",3512.0,1168.0,331981.810256,384679.597096,173527.505993,211152.091103,3,4680.0
1,0.753188,"frozenset({'Gender=Man', 'Asian=0', 'White=1', 'Black=0'})",3501.0,1165.0,332122.686241,384808.381891,173794.256652,211014.125239,4,4666.0
2,0.763519,"frozenset({'Gender=Man', 'White=1'})",3560.0,1170.0,332456.905497,384593.409270,173819.338462,210774.070808,2,4730.0
3,0.761098,"frozenset({'Gender=Man', 'White=1', 'Black=0'})",3548.0,1167.0,332615.216331,384758.155862,174086.382177,210671.773686,3,4715.0
4,0.954157,frozenset({'Gender=Man'}),4402.0,1509.0,336704.732363,390120.222399,180883.004639,209237.217760,1,5911.0
5,0.940113,"frozenset({'Gender=Man', 'Black=0'})",4326.0,1498.0,335658.379464,389036.590846,181510.086782,207526.504064,2,5824.0
6,0.786118,"frozenset({'Asian=0', 'White=1'})",3660.0,1210.0,326671.150924,377957.460383,171540.661157,206416.799225,2,4870.0
7,0.783697,"frozenset({'Asian=0', 'White=1', 'Black=0'})",3648.0,1207.0,326842.036457,378142.462993,171793.191384,206349.271610,3,4855.0
8,0.795319,frozenset({'White=1'}),3715.0,1212.0,327094.427035,377750.078869,171825.659241,205924.419629,1,4927.0
9,0.792736,"frozenset({'White=1', 'Black=0'})",3702.0,1209.0,327280.355121,377966.219071,172078.478908,205887.740163,2,4911.0


In [2]:
df_t = pd.read_csv("outputs/so/subpopulations_and_treatments.csv")
df_t

,itemset,treatment
0,"frozenset({'Gender=Man', 'Asian=0', 'White=1'})","{'att': 'Developer, back-end', 'value': 1}"
1,"frozenset({'Gender=Man', 'Asian=0', 'White=1', 'Black=0'})","{'att': 'Developer, back-end', 'value': 1}"
2,"frozenset({'Gender=Man', 'White=1'})","{'att': 'Developer, back-end', 'value': 1}"
3,"frozenset({'Gender=Man', 'White=1', 'Black=0'})","{'att': 'Developer, back-end', 'value': 1}"
4,frozenset({'Gender=Man'}),"{'att': 'Developer, back-end', 'value': 1}"
5,"frozenset({'Gender=Man', 'Black=0'})","{'att': 'Developer, back-end', 'value': 1}"
6,"frozenset({'Asian=0', 'White=1'})","{'att': 'Developer, back-end', 'value': 1}"
7,"frozenset({'Asian=0', 'White=1', 'Black=0'})","{'att': 'Developer, back-end', 'value': 1}"
8,frozenset({'White=1'}),"{'att': 'Developer, back-end', 'value': 1}"
9,"frozenset({'White=1', 'Black=0'})","{'att': 'Developer, back-end', 'value': 1}"


In [3]:
df_final = pd.read_csv("outputs/so/find_k/5_0.5_9e-05.csv")
df_final

,itemset,treatment,ate1,ate2,iscore,size_itemset,size_group1,size_group2,support,ni_score,p_v1,p_v2,utility,std,diff_means
0,{'Black': 0},"{'Developer, back-end': 1}",88830.713701,-31867.131422,120697.845123,23384,4538,1561,0.989464,0.999981,[0.01315214],[0.06450854],0.989445,594686.184114,201614.962762
1,{'Gender': 'Man'},"{'Developer, back-end': 1}",79005.316335,-32175.411159,111180.727494,22707,4402,1509,0.960818,0.999955,[0.03295728],[0.06856103],0.960774,604909.502986,209237.217760
2,{'Asian': 0},"{'Developer, back-end': 1}",79150.613543,-30361.567942,109512.181485,22090,4348,1542,0.934710,0.999948,[0.02512849],[0.08034475],0.934661,580861.545899,196023.485700
3,"{'Gender': 'Man', 'Black': 0}","{'Developer, back-end': 1}",86121.016883,-32329.623483,118450.640366,22473,4326,1498,0.950916,0.999977,[0.02124827],[0.06925206],0.950894,604104.982832,207526.504064
4,"{'Asian': 0, 'Black': 0}","{'Developer, back-end': 1}",86241.168795,-30337.081487,116578.250282,21850,4266,1530,0.924555,0.999972,[0.01565449],[0.08299544],0.924529,579814.490071,194277.644342


In [4]:
df_scores = pd.read_csv("outputs/so/scores/5_0.5_9e-05.csv")
df_scores

,utility_sum,utility,intersection_sum,final_intersection,score
0,0.989445,0.989445,0,1.000000,0.994722
1,1.950219,0.975110,22473,0.759740,0.867425
2,2.884880,0.961627,65570,0.688438,0.825033
3,3.835774,0.958943,131538,0.648429,0.803686
4,4.760303,0.952061,217282,0.628324,0.790192


# Comparison to DivExplorer:

0. Define DivExplorer run to search for every itemset and find group1's and group2's outcome average, and divergity=Group1_income - group2_income
1. for each t:
    1. run Divexplorer on the treated in Group1+Group2
    2. get the top 5 diff means groups
2. combine all the groups + treatments
3. return the top 5 diff means

In [5]:
divexplorer_df = pd.read_csv("outputs/so/divexplorer_run.csv")
divexplorer_df

,Unnamed: 0,support,itemset,GROUP_1,GROUP_2,ConvertedCompYearly,ConvertedCompYearly_group1,ConvertedCompYearly_group2,ConvertedCompYearly_div,length,support_count,treatment_att,treatment_val
0,0,0.961169,"frozenset({'Gender=Man', 'Black=0'})",1855.0,646.0,162429.786200,438228.692722,163122.343653,275106.349069,2,10594.0,"Developer, back-end",1
1,1,0.970876,frozenset({'Gender=Man'}),1879.0,652.0,161648.153070,435402.006386,162609.759202,272792.247184,1,10701.0,"Developer, back-end",1
2,2,0.989657,frozenset({'Black=0'}),1915.0,664.0,160595.408232,432153.206266,160881.524096,271271.682170,1,10908.0,"Developer, back-end",1
3,3,0.894121,"frozenset({'Asian=0', 'Gender=Man', 'Black=0'})",1741.0,635.0,163020.075495,428406.029294,164071.905512,264334.123782,3,9855.0,"Developer, back-end",1
4,4,0.903375,"frozenset({'Gender=Man', 'Asian=0'})",1764.0,641.0,162213.151050,425663.856009,163541.636505,262122.219504,2,9957.0,"Developer, back-end",1


Got the same set!

# Heart data

Want to compare the causes for different Chest Pain Types, Will define: <br>
Group1 = Typical Angina <br>
Group2 = Asymptomatic <br>
The subpopulations: {"Sex", "Age"} <br>
The treatments: {"Cholesterol", "ExerciseAngina"} <br>

In [6]:
df_t = pd.read_csv("outputs/heart/subpopulations_and_treatments.csv")
df_t

,itemset,treatment
0,"frozenset({'Sex=M', 'Age=40-50'})","{'att': 'Cholesterol', 'value': '180-240'}"
1,frozenset({'Age=40-50'}),"{'att': 'Cholesterol', 'value': '180-240'}"
2,frozenset({'Sex=M'}),"{'att': 'Cholesterol', 'value': '180-240'}"
3,"frozenset({'Sex=M', 'Age=30-40'})","{'att': 'Cholesterol', 'value': '180-240'}"
4,frozenset({'Age=30-40'}),"{'att': 'Cholesterol', 'value': '180-240'}"
5,"frozenset({'Sex=M', 'Age=50-60'})","{'att': 'Cholesterol', 'value': '120-180'}"
6,frozenset({'Sex=F'}),"{'att': 'Cholesterol', 'value': '180-240'}"
7,"frozenset({'Sex=F', 'Age=30-40'})","{'att': 'Cholesterol', 'value': '180-240'}"


In [7]:
df_final = pd.read_csv("outputs/heart/find_k/5_0.5_9e-05.csv")
df_final

,itemset,treatment,ate1,ate2,iscore,size_itemset,size_group1,size_group2,support,ni_score,p_v1,p_v2,utility,std,diff_means
0,{'Sex': 'M'},{'Cholesterol': '180-240'},-0.333333,-0.113419,0.219915,725,36,426,0.789760,1.979212e-05,[0.04994953],[0.00480008],1.563104e-05,0.482337,-0.300861
1,"{'Sex': 'F', 'Age': '30-40'}",{'Cholesterol': '180-240'},-0.333333,-0.250000,0.083333,63,7,30,0.068627,7.499972e-06,[0.28559094],[0.24667273],5.147040e-07,0.481452,-0.557143
2,{'Sex': 'F'},{'Cholesterol': '180-240'},-0.200000,-0.192009,0.007991,193,10,70,0.210240,7.192006e-07,[0.34659351],[0.17026427],1.512045e-07,0.438123,-0.457143
3,"{'Sex': 'M', 'Age': '40-50'}",{'Cholesterol': '180-240'},-0.666667,-0.164276,0.502390,159,7,85,0.173203,4.521410e-05,[0.28559094],[0.11899049],7.831201e-06,0.499990,-0.157983
4,"{'Sex': 'M', 'Age': '30-40'}",{'Cholesterol': '180-240'},-0.444444,-0.047059,0.397386,235,18,144,0.255991,3.576407e-05,[0.06277196],[0.44890817],9.155289e-06,0.455419,-0.333333


In [8]:
df_scores = pd.read_csv("outputs/heart/scores/5_0.5_9e-05.csv")
df_scores

,utility_sum,utility,intersection_sum,final_intersection,score
0,0.000016,0.000016,0,1.000000,0.500008
1,0.000016,0.000008,0,1.000000,0.500004
2,0.000016,0.000005,63,0.990345,0.495175
3,0.000024,0.000006,222,0.980862,0.490434
4,0.000033,0.000007,457,0.974786,0.487396


In [9]:
divexplorer_df = pd.read_csv("outputs/heart/divexplorer_run.csv")
divexplorer_df

,Unnamed: 0,support,itemset,GROUP_1,GROUP_2,HeartDisease,HeartDisease_group1,HeartDisease_group2,HeartDisease_div,length,support_count,treatment_att,treatment_val
0,0,0.387640,"frozenset({'Sex=M', 'Age=30-40'})",3,51,0.869565,1.0,0.921569,0.078431,2,69.0,Cholesterol,<120
1,1,0.938202,frozenset({'Sex=M'}),4,122,0.874251,1.0,0.926230,0.073770,1,167.0,Cholesterol,<120
2,2,0.432584,frozenset({'Age=30-40'}),4,57,0.870130,1.0,0.929825,0.070175,1,77.0,Cholesterol,<120
3,3,0.112360,frozenset({'Age=40-50'}),1,15,0.900000,1.0,0.933333,0.066667,1,20.0,Cholesterol,<120
4,4,0.112360,"frozenset({'Age=40-50', 'Sex=M'})",1,15,0.900000,1.0,0.933333,0.066667,2,20.0,Cholesterol,<120
